In [12]:
import numpy as np
import pandas as pd
from main import sskpy

prvi = "This is a very long string, just to test how fast this implementation " \
        "oook like ttime, unless you're" \
        " running this in a potato pc"
drugi="dfgbs sfhsbgs vsjh"
a=0.8
rez=sskpy(prvi, prvi, 9,a )
print(rez)

175922.15625


In [2]:
def similarity_matrix(first_dataset, second_dataset, similarity_function, symmetrical=False):
    """
    Calculate the similarity matrix between elements of two datasets using a similarity function.

    Args:
    - first_dataset: List or array-like, the first dataset
    - second_dataset: List or array-like, the second dataset
    - similarity_function: Function, the similarity function that takes two elements as arguments

    Returns:
    - similarity_matrix: NumPy ndarray, the similarity matrix
    """
    if(symmetrical):
        size = len(first_dataset)
        similarity_matrix = [[0.0] * size for _ in range(size)]

        for i in range(size):
            for j in range(i, size):
                value =similarity_function(first_dataset[i], second_dataset[j])
                similarity_matrix[i][j] = value
                similarity_matrix[j][i] = value 
    else:
        similarity_matrix = [[similarity_function(x, y) for y in second_dataset] for x in first_dataset]
    
    return similarity_matrix

In [21]:
from main import sskpy
def ssk(a, b, k, lambd):
    return sskpy(a,b,k,lambd)

def ssk_scaled(a, b, k, lambd):    #accounts for document length
    len_a=len(a)
    len_b=len(b)
    return sskpy(a,b,k,lambd)/(max(len_a, len_b))

In [14]:
def ssk_partial(ka, Lambda):
    return lambda a,b: ssk(a,b,k=ka, lambd=Lambda)

def ssk_scaled_partial(ka, Lambda):
    return lambda a,b: ssk_scaled(a,b,k=ka, lambd=Lambda)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support


def WK_SVM(X_train, y_train, X_test, y_test):  #a multi-class classifier

    '''calculates f1, precision, and recall for a SVM classifer using linear tfidf mapping
    Args:
    - X_train,y_train, X_test, y_test
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel, Class, F1, Precision , Recall
    '''
    vectorizer = TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True, norm='l2',
                                 analyzer='word', stop_words='english')

    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(X_train_tfidf, y_train)

    y_pred = svm_classifier.predict(X_test_tfidf)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
    results_df = pd.DataFrame({
    'Kernel' : 'WK',
    'Class': range(len(precision)),
    'F1-Score': f1,
    'Precision': precision,
    'Recall': recall,
    })
    return results_df


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

def NGK_SVM(X_train, y_train, X_test, y_test, k):
    '''calculates f1, precision, and recall for a SVM classifer using linear n-gram mapping
    Args:
    - X_train,y_train, X_test, y_test, k=lenthg of the ngrams
    Returns:
    - f1, precision, recall: for each of the classes in form of a pandas dataframe w columns Kernel,k, Class, F1, Precision , Recall
    '''
    ngram_range = (k, k)
    vectorizer = CountVectorizer(analyzer='char', ngram_range=ngram_range)

    x_train_ngrams = normalize(vectorizer.fit_transform(X_train), norm='l2')  #ngram vectors normalised to l2 norm
    x_test_ngrams = normalize(vectorizer.transform(X_test), norm='l2')

    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(x_train_ngrams, y_train)

    y_pred = svm_classifier.predict(x_test_ngrams)

    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

    results_df = pd.DataFrame({
        'Kernel':'NGK',
        'k':k,
        'Class': range(len(precision)),
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall
        })
    return results_df